<a href="https://colab.research.google.com/github/maclandrol/cours-ia-med/blob/master/05_TorchXRayVision_Complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. TorchXRayVision - Analyse Complète de Radiographies Thoraciques

**Enseignant:** Emmanuel Noutahi, PhD

---

**Objectif:** Maîtriser l'analyse complète de radiographies thoraciques avec TorchXRayVision.

**Applications pratiques :**
- Classification multi-pathologies avec 18 conditions détectables
- Comparaison de modèles pré-entraînés (DenseNet-All, CheXpert, NIH)
- Localisation spatiale avec cartes d'activation Grad-CAM
- Génération automatique de rapports cliniques
- Intégration dans workflows hospitaliers

**Important:** Ce cours couvre l'ensemble du pipeline d'analyse radiologique avec TorchXRayVision.

## Installation et Configuration

In [ ]:
# Installation des bibliothèques pour analyse radiologique complète
!pip install torchxrayvision torch torchvision matplotlib numpy pandas seaborn scikit-learn -q
!pip install opencv-python pillow scipy -q

import torch
import torchvision
import torchxrayvision as xrv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from scipy import ndimage
import warnings
warnings.filterwarnings('ignore')

# Configuration système
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(42)
np.random.seed(42)

print(f"Dispositif utilisé: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print(f"Version TorchXRayVision: {xrv.__version__}")
print("Configuration TorchXRayVision terminée.")

## 1. Chargement des Modèles Multi-Dataset

TorchXRayVision propose plusieurs modèles spécialisés entraînés sur différents datasets hospitaliers.

In [ ]:
# Chargement des modèles TorchXRayVision
print("=== CHARGEMENT DES MODÈLES TORCHXRAYVISION ===")

def load_xrayvision_models(device):
    """
    Charge les modèles TorchXRayVision disponibles pour comparaison
    """
    models = {}
    model_info = {}
    
    # Modèle universel (tous datasets combinés)
    try:
        print("Chargement modèle DenseNet-All...")
        models['densenet_all'] = xrv.models.DenseNet(weights="densenet121-res224-all")
        models['densenet_all'].eval().to(device)
        
        model_info['densenet_all'] = {
            'name': 'DenseNet-All',
            'description': 'Entraîné sur tous les datasets disponibles',
            'datasets': 'CheXpert + NIH + MIMIC + PC + Padchest',
            'pathologies': 18,
            'specialty': 'Généraliste haute performance'
        }
        print("✓ DenseNet-All chargé")
    except Exception as e:
        print(f"Erreur DenseNet-All: {e}")
    
    # Modèle CheXpert (Stanford)
    try:
        print("Chargement modèle CheXpert...")
        models['chexpert'] = xrv.models.DenseNet(weights="densenet121-res224-chex")
        models['chexpert'].eval().to(device)
        
        model_info['chexpert'] = {
            'name': 'CheXpert',
            'description': 'Modèle Stanford CheXpert',
            'datasets': 'CheXpert (224k images Stanford)',
            'pathologies': 14,
            'specialty': 'Pathologies thoraciques variées'
        }
        print("✓ CheXpert chargé")
    except Exception as e:
        print(f"Erreur CheXpert: {e}")
    
    # Modèle NIH
    try:
        print("Chargement modèle NIH...")
        models['nih'] = xrv.models.DenseNet(weights="densenet121-res224-nih")
        models['nih'].eval().to(device)
        
        model_info['nih'] = {
            'name': 'NIH ChestX-ray8',
            'description': 'Modèle NIH National Institutes of Health',
            'datasets': 'NIH ChestX-ray8 (112k images)',
            'pathologies': 14,
            'specialty': '14 pathologies principales'
        }
        print("✓ NIH chargé")
    except Exception as e:
        print(f"Erreur NIH: {e}")
    
    return models, model_info

# Chargement des modèles
models_dict, models_info = load_xrayvision_models(device)

print(f"\nModèles chargés: {len(models_dict)}")
for model_key, info in models_info.items():
    if model_key in models_dict:
        print(f"• {info['name']}: {info['description']}")

# Récupération des pathologies détectables
if models_dict:
    primary_model = list(models_dict.values())[0]
    pathologies = primary_model.pathologies
    
    print(f"\nPathologies détectables ({len(pathologies)}):")
    for i, pathology in enumerate(pathologies, 1):
        print(f"{i:2d}. {pathology}")
else:
    print("Aucun modèle disponible")

## 2. Création de Cas Cliniques Réalistes

Créons une bibliothèque de radiographies représentant différents cas cliniques pour évaluer les modèles.

In [ ]:
# Création de radiographies synthétiques réalistes
print("=== CRÉATION DE CAS CLINIQUES RÉALISTES ===")

def create_synthetic_chest_xray(pathology_type, severity='moderate', size=224):
    """
    Crée des radiographies synthétiques avec pathologies spécifiques
    
    pathology_type: 'normal', 'pneumonia', 'cardiomegaly', 'pneumothorax', 
                   'pleural_effusion', 'atelectasis', 'nodule'
    severity: 'mild', 'moderate', 'severe'
    """
    img = np.zeros((size, size), dtype=np.float32)
    center_x, center_y = size // 2, size // 2
    
    # Structure anatomique de base
    
    # Parenchyme pulmonaire
    lung_intensity = 0.3
    
    # Poumon droit
    right_lung_y, right_lung_x = int(0.4 * size), int(0.3 * size)
    right_lung_h, right_lung_w = int(0.25 * size), int(0.12 * size)
    rr1, cc1 = np.meshgrid(np.arange(size), np.arange(size), indexing='ij')
    right_mask = ((rr1 - right_lung_y)**2 / right_lung_h**2 + 
                 (cc1 - right_lung_x)**2 / right_lung_w**2) <= 1
    img[right_mask] = lung_intensity
    
    # Poumon gauche (légèrement plus petit anatomiquement)
    left_lung_y, left_lung_x = int(0.4 * size), int(0.7 * size)
    left_lung_h, left_lung_w = int(0.23 * size), int(0.12 * size)
    left_mask = ((rr1 - left_lung_y)**2 / left_lung_h**2 + 
                (cc1 - left_lung_x)**2 / left_lung_w**2) <= 1
    img[left_mask] = lung_intensity
    
    # Silhouette cardiaque (taille variable selon pathologie)
    heart_size_factor = 1.0
    if pathology_type == 'cardiomegaly':
        heart_size_factor = {'mild': 1.3, 'moderate': 1.6, 'severe': 2.0}[severity]
    
    heart_y, heart_x = int(0.6 * size), int(0.48 * size)
    heart_h = int(0.08 * size * heart_size_factor)
    heart_w = int(0.1 * size * heart_size_factor)
    heart_mask = ((rr1 - heart_y)**2 / heart_h**2 + 
                 (cc1 - heart_x)**2 / heart_w**2) <= 1
    img[heart_mask] = 0.6
    
    # Rachis et structures médiastinales
    spine_width = int(0.02 * size)
    img[int(0.1*size):int(0.9*size), center_x-spine_width:center_x+spine_width] = 0.8
    
    # Gril costal
    for rib_level in range(8):
        rib_y = int(0.15 * size) + rib_level * int(0.06 * size)
        for x in range(int(0.05 * size), int(0.95 * size)):
            if x < center_x:
                curve = int(0.02 * size * np.sin(np.pi * (x - 0.05 * size) / (0.45 * size)))
            else:
                curve = int(0.02 * size * np.sin(np.pi * (0.95 * size - x) / (0.45 * size)))
            y_rib = rib_y + curve
            if 0 <= y_rib < size and 0 <= x < size:
                img[y_rib:y_rib+1, x:x+1] = 0.7
    
    # Pathologies spécifiques
    
    if pathology_type == 'pneumonia':
        # Consolidation pneumonique avec bronchogramme aérique
        consolidation_intensity = {'mild': 0.7, 'moderate': 0.8, 'severe': 0.9}[severity]
        consolidation_size = {'mild': 0.06, 'moderate': 0.1, 'severe': 0.15}[severity]
        
        cons_y, cons_x = int(0.35 * size), int(0.25 * size)
        cons_h = cons_w = int(consolidation_size * size)
        consolidation_mask = ((rr1 - cons_y)**2 / cons_h**2 + 
                             (cc1 - cons_x)**2 / cons_w**2) <= 1
        img[consolidation_mask] = consolidation_intensity
        
        # Bronchogramme aérique (tubulures aériennes visibles)
        for i in range(3):
            broncho_y = cons_y + (i-1) * 4
            broncho_x_start, broncho_x_end = cons_x - 8, cons_x + 8
            img[broncho_y:broncho_y+1, broncho_x_start:broncho_x_end] = 0.4
    
    elif pathology_type == 'pneumothorax':
        # Ligne pleurale avec décollement
        pleural_line_x = {'mild': int(0.18 * size), 'moderate': int(0.15 * size), 
                         'severe': int(0.12 * size)}[severity]
        pleural_start_y, pleural_end_y = int(0.2 * size), int(0.7 * size)
        
        # Ligne de décollement pleural
        img[pleural_start_y:pleural_end_y, pleural_line_x:pleural_line_x+2] = 0.95
        
        # Zone de décollement (hyperclarté pneumothoracique)
        img[pleural_start_y:pleural_end_y, int(0.05*size):pleural_line_x] = 0.1
        
        # Rétraction pulmonaire
        retraction_factor = {'mild': 0.8, 'moderate': 0.6, 'severe': 0.4}[severity]
        affected_region = img[pleural_start_y:pleural_end_y, pleural_line_x:int(0.5*size)]
        img[pleural_start_y:pleural_end_y, pleural_line_x:int(0.5*size)] = affected_region * retraction_factor
    
    elif pathology_type == 'pleural_effusion':
        # Épanchement pleural avec ligne de Damoiseau
        effusion_height = {'mild': int(0.12 * size), 'moderate': int(0.2 * size), 
                          'severe': int(0.3 * size)}[severity]
        effusion_start_y = int(0.75 * size) - effusion_height
        effusion_end_y = int(0.75 * size)
        
        # Opacité hydrique homogène
        for y in range(effusion_start_y, effusion_end_y):
            # Ligne concave caractéristique (signe de Damoiseau)
            curve_offset = int(0.04 * size * np.sin(np.pi * (y - effusion_start_y) / effusion_height))
            x_limit = int(0.5 * size) - curve_offset
            img[y, int(0.1*size):x_limit] = 0.85
    
    elif pathology_type == 'atelectasis':
        # Atélectasie avec déplacement des scissures
        atelectasis_intensity = {'mild': 0.65, 'moderate': 0.75, 'severe': 0.85}[severity]
        
        # Zone atélectasiée
        atel_y, atel_x = int(0.3 * size), int(0.3 * size)
        atel_h, atel_w = int(0.08 * size), int(0.15 * size)
        atelectasis_mask = ((rr1 - atel_y)**2 / atel_h**2 + 
                           (cc1 - atel_x)**2 / atel_w**2) <= 1
        img[atelectasis_mask] = atelectasis_intensity
        
        # Déplacement des structures adjacentes
        displacement = {'mild': 2, 'moderate': 4, 'severe': 6}[severity]
        # Simulation simple du déplacement
        shifted_region = np.roll(img[int(0.25*size):int(0.4*size), int(0.2*size):int(0.5*size)], 
                               displacement, axis=1)
        img[int(0.25*size):int(0.4*size), int(0.2*size):int(0.5*size)] = shifted_region
    
    elif pathology_type == 'nodule':
        # Nodule pulmonaire
        nodule_size = {'mild': 3, 'moderate': 6, 'severe': 10}[severity]
        nodule_intensity = 0.8
        
        # Position aléatoire dans parenchyme pulmonaire
        nodule_y = int(0.4 * size) + np.random.randint(-int(0.1*size), int(0.1*size))
        nodule_x = int(0.3 * size) + np.random.randint(-int(0.05*size), int(0.05*size))
        
        nodule_mask = ((rr1 - nodule_y)**2 + (cc1 - nodule_x)**2) <= nodule_size**2
        img[nodule_mask] = nodule_intensity
        
        # Halo éventuel (inflammation périphérique)
        if severity == 'severe':
            halo_mask = ((rr1 - nodule_y)**2 + (cc1 - nodule_x)**2) <= (nodule_size + 3)**2
            halo_mask &= ~nodule_mask
            img[halo_mask] = 0.5
    
    # Post-traitement réaliste
    
    # Bruit quantique radiographique
    noise = np.random.normal(0, 0.03, img.shape)
    img += noise
    
    # Flou de mouvement léger
    img = ndimage.gaussian_filter(img, sigma=0.6)
    
    # Normalisation finale
    img = np.clip(img, 0, 1)
    
    return img

# Création de la bibliothèque de cas cliniques
clinical_cases = {
    'normal': {
        'image': create_synthetic_chest_xray('normal'),
        'description': 'Radiographie thoracique normale',
        'expected_pathologies': [],
        'clinical_context': 'Bilan systématique, patient asymptomatique, 35 ans'
    },
    'pneumonia_lobaire': {
        'image': create_synthetic_chest_xray('pneumonia', 'moderate'),
        'description': 'Pneumonie lobaire inférieure droite',
        'expected_pathologies': ['Pneumonia', 'Infiltration'],
        'clinical_context': 'Fièvre 39°C, toux productive purulente, frissons, dyspnée'
    },
    'cardiomegalie_moderee': {
        'image': create_synthetic_chest_xray('cardiomegaly', 'moderate'),
        'description': 'Cardiomégalie modérée par insuffisance cardiaque',
        'expected_pathologies': ['Cardiomegaly', 'Enlarged Cardiomediastinum'],
        'clinical_context': 'Dyspnée d\'effort NYHA II, œdèmes malléolaires, orthopnée'
    },
    'pneumothorax_spontane': {
        'image': create_synthetic_chest_xray('pneumothorax', 'moderate'),
        'description': 'Pneumothorax spontané partiel droit',
        'expected_pathologies': ['Pneumothorax'],
        'clinical_context': 'Douleur thoracique latérale droite brutale, dyspnée, homme jeune'
    },
    'epanchement_pleural': {
        'image': create_synthetic_chest_xray('pleural_effusion', 'moderate'),
        'description': 'Épanchement pleural liquidien abondant',
        'expected_pathologies': ['Pleural Effusion'],
        'clinical_context': 'Dyspnée progressive, douleur pleurale, matité à la percussion'
    },
    'atelectasie_lobaire': {
        'image': create_synthetic_chest_xray('atelectasis', 'moderate'),
        'description': 'Atélectasie lobaire par obstruction bronchique',
        'expected_pathologies': ['Atelectasis'],
        'clinical_context': 'Toux sèche persistante, dyspnée, antécédents tabagiques'
    }
}

print(f"Bibliothèque de cas cliniques créée: {len(clinical_cases)} cas")

# Visualisation de la bibliothèque
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Bibliothèque de Cas Cliniques - TorchXRayVision', fontsize=16, fontweight='bold')

axes = axes.flatten()
for i, (case_name, case_data) in enumerate(clinical_cases.items()):
    if i < len(axes):
        axes[i].imshow(case_data['image'], cmap='gray')
        axes[i].set_title(f"{case_data['description']}", fontweight='bold', fontsize=11)
        axes[i].axis('off')
        
        # Contexte clinique
        axes[i].text(0.02, 0.02, f"Contexte: {case_data['clinical_context'][:50]}...", 
                    transform=axes[i].transAxes, fontsize=8,
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.9),
                    verticalalignment='bottom')

plt.tight_layout()
plt.show()

print("Cas cliniques prêts pour analyse multi-modèles.")

## 3. Preprocessing et Classification Multi-Modèles

Analysons nos cas cliniques avec tous les modèles disponibles et comparons leurs performances.

In [ ]:
# Preprocessing standardisé et classification multi-modèles
print("=== PREPROCESSING ET CLASSIFICATION MULTI-MODÈLES ===")

def preprocess_for_xrayvision(image_array):
    """
    Preprocessing standardisé pour TorchXRayVision
    """
    # Normalisation
    if image_array.max() > 1:
        img_norm = xrv.datasets.normalize(image_array, 255)
    else:
        img_norm = image_array
    
    # Ajout dimension canal
    img_channel = img_norm[None, ...]
    
    # Transformations TorchXRayVision
    transform = torchvision.transforms.Compose([
        xrv.datasets.XRayCenterCrop(),
        xrv.datasets.XRayResizer(224)
    ])
    
    img_transformed = transform(img_channel)
    
    # Conversion en tenseur PyTorch
    img_tensor = torch.from_numpy(img_transformed).float()
    img_batch = img_tensor.unsqueeze(0)
    
    return img_batch, img_transformed

def classify_with_all_models(image_tensor, models_dict, pathologies, threshold=0.5):
    """
    Classification avec tous les modèles disponibles
    """
    results = {}
    
    for model_name, model in models_dict.items():
        model.eval()
        image_tensor_device = image_tensor.to(device)
        
        with torch.no_grad():
            outputs = model(image_tensor_device)
            probabilities = torch.sigmoid(outputs).cpu().numpy().squeeze()
            
            # Détections positives
            positive_indices = np.where(probabilities > threshold)[0]
            positive_findings = [(pathologies[i], probabilities[i]) for i in positive_indices]
            
            # Stockage des résultats
            results[model_name] = {
                'probabilities': probabilities,
                'positive_findings': positive_findings,
                'max_probability': np.max(probabilities),
                'mean_probability': np.mean(probabilities),
                'num_detections': len(positive_findings)
            }
    
    return results

def analyze_model_consensus(case_results, expected_pathologies):
    """
    Analyse du consensus entre modèles
    """
    # Collecte de toutes les détections
    all_detections = {}
    
    for model_name, result in case_results.items():
        for pathology, prob in result['positive_findings']:
            if pathology not in all_detections:
                all_detections[pathology] = []
            all_detections[pathology].append((model_name, prob))
    
    # Calcul du consensus
    consensus_findings = []
    for pathology, detections in all_detections.items():
        consensus_level = len(detections) / len(case_results)
        avg_probability = np.mean([prob for _, prob in detections])
        
        consensus_findings.append({
            'pathology': pathology,
            'consensus_level': consensus_level,
            'avg_probability': avg_probability,
            'detecting_models': [model for model, _ in detections],
            'num_models': len(detections)
        })
    
    # Tri par niveau de consensus
    consensus_findings.sort(key=lambda x: (x['consensus_level'], x['avg_probability']), reverse=True)
    
    # Évaluation par rapport aux pathologies attendues
    evaluation = {
        'true_positives': 0,
        'false_positives': 0,
        'false_negatives': len(expected_pathologies),
        'detected_expected': [],
        'missed_expected': expected_pathologies.copy(),
        'unexpected_detections': []
    }
    
    for finding in consensus_findings:
        pathology = finding['pathology']
        is_expected = any(exp.lower() in pathology.lower() or 
                         pathology.lower() in exp.lower() 
                         for exp in expected_pathologies)
        
        if is_expected:
            evaluation['true_positives'] += 1
            evaluation['false_negatives'] -= 1
            evaluation['detected_expected'].append(pathology)
            # Retirer de missed_expected
            for exp in expected_pathologies:
                if exp.lower() in pathology.lower() or pathology.lower() in exp.lower():
                    if exp in evaluation['missed_expected']:
                        evaluation['missed_expected'].remove(exp)
                    break
        else:
            evaluation['false_positives'] += 1
            evaluation['unexpected_detections'].append(pathology)
    
    return consensus_findings, evaluation

# Analyse de tous les cas cliniques
print("\nAnalyse en cours de tous les cas cliniques...")

comprehensive_analysis = {}

if models_dict and pathologies:
    for case_name, case_data in clinical_cases.items():
        print(f"\nAnalyse du cas: {case_name}")
        
        # Preprocessing
        processed_tensor, processed_display = preprocess_for_xrayvision(case_data['image'])
        
        # Classification multi-modèles
        case_results = classify_with_all_models(processed_tensor, models_dict, pathologies)
        
        # Analyse du consensus
        consensus_findings, evaluation = analyze_model_consensus(
            case_results, case_data['expected_pathologies']
        )
        
        # Stockage des résultats
        comprehensive_analysis[case_name] = {
            'case_info': case_data,
            'processed_image': processed_display,
            'model_results': case_results,
            'consensus': consensus_findings,
            'evaluation': evaluation
        }
        
        # Affichage des résultats principaux
        print(f"  Description: {case_data['description']}")
        print(f"  Pathologies attendues: {case_data['expected_pathologies']}")
        print(f"  Consensus détections (>50% modèles):")
        
        high_consensus = [f for f in consensus_findings if f['consensus_level'] > 0.5]
        if high_consensus:
            for finding in high_consensus[:3]:  # Top 3
                print(f"    • {finding['pathology']}: {finding['consensus_level']*100:.0f}% modèles, prob={finding['avg_probability']:.3f}")
        else:
            print(f"    • Aucune détection consensuelle")
        
        print(f"  Évaluation: TP={evaluation['true_positives']}, FP={evaluation['false_positives']}, FN={evaluation['false_negatives']}")
    
    print(f"\nAnalyse complétée pour {len(comprehensive_analysis)} cas.")
else:
    print("Erreur: Modèles ou pathologies non disponibles")
    comprehensive_analysis = {}

## 4. Localisation Spatiale avec Grad-CAM

Utilisons Grad-CAM pour visualiser les régions importantes identifiées par les modèles.

In [ ]:
# Implémentation de Grad-CAM pour localisation spatiale
print("=== LOCALISATION SPATIALE AVEC GRAD-CAM ===")

class GradCAM:
    """
    Implémentation de Gradient-weighted Class Activation Mapping (Grad-CAM)
    pour localisation des pathologies dans les radiographies
    """
    
    def __init__(self, model, target_layer_name='features'):
        self.model = model
        self.target_layer_name = target_layer_name
        self.gradients = None
        self.activations = None
        
        # Enregistrement des hooks
        self.hook_layers()
    
    def hook_layers(self):
        """
        Enregistre les hooks pour capturer gradients et activations
        """
        def backward_hook(module, grad_input, grad_output):
            self.gradients = grad_output[0]
        
        def forward_hook(module, input, output):
            self.activations = output
        
        # Recherche de la couche cible
        target_layer = None
        for name, module in self.model.named_modules():
            if self.target_layer_name in name:
                target_layer = module
                break
        
        if target_layer is None:
            # Fallback: utiliser la dernière couche convolutionnelle
            for name, module in reversed(list(self.model.named_modules())):
                if isinstance(module, torch.nn.Conv2d):
                    target_layer = module
                    break
        
        if target_layer is not None:
            target_layer.register_forward_hook(forward_hook)
            target_layer.register_full_backward_hook(backward_hook)
    
    def generate_cam(self, input_image, class_idx=None):
        """
        Génère la carte d'activation pour une classe spécifique
        """
        self.model.eval()
        input_image = input_image.requires_grad_()
        
        # Forward pass
        output = self.model(input_image)
        
        # Si classe non spécifiée, prendre celle avec probabilité max
        if class_idx is None:
            class_idx = torch.argmax(torch.sigmoid(output), dim=1)
        
        # Backward pass
        self.model.zero_grad()
        class_score = output[0, class_idx]
        class_score.backward(retain_graph=True)
        
        # Génération de la carte Grad-CAM
        if self.gradients is not None and self.activations is not None:
            # Moyennage spatial des gradients
            alpha = torch.mean(self.gradients, dim=[2, 3], keepdim=True)
            
            # Combinaison pondérée des cartes d'activation
            cam = torch.sum(alpha * self.activations, dim=1, keepdim=True)
            
            # Application de ReLU
            cam = torch.clamp(cam, min=0)
            
            # Normalisation
            cam = cam / (torch.max(cam) + 1e-8)
            
            # Redimensionnement à la taille de l'image d'entrée
            cam_resized = torch.nn.functional.interpolate(
                cam, size=input_image.shape[2:], mode='bilinear', align_corners=False
            )
            
            return cam_resized.squeeze().cpu().numpy()
        
        return None

def generate_gradcam_visualization(model, image_tensor, pathologies, top_k=3):
    """
    Génère les visualisations Grad-CAM pour les pathologies les plus probables
    """
    # Prédiction pour identifier les pathologies principales
    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor.to(device))
        probabilities = torch.sigmoid(outputs).cpu().numpy().squeeze()
    
    # Sélection des pathologies les plus probables
    top_indices = np.argsort(probabilities)[-top_k:][::-1]
    
    # Génération des cartes Grad-CAM
    gradcam = GradCAM(model)
    cam_results = []
    
    for idx in top_indices:
        pathology = pathologies[idx]
        probability = probabilities[idx]
        
        # Génération de la carte CAM
        cam_map = gradcam.generate_cam(image_tensor.to(device), class_idx=idx)
        
        if cam_map is not None:
            cam_results.append({
                'pathology': pathology,
                'probability': probability,
                'cam_map': cam_map,
                'class_idx': idx
            })
    
    return cam_results

def visualize_gradcam_results(image, cam_results, title="Localisation Grad-CAM"):
    """
    Visualise les résultats Grad-CAM
    """
    num_results = len(cam_results)
    if num_results == 0:
        print("Aucun résultat Grad-CAM à afficher")
        return
    
    fig, axes = plt.subplots(1, num_results + 1, figsize=(4 * (num_results + 1), 4))
    if num_results == 0:
        axes = [axes]
    
    # Image originale
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Image Originale', fontweight='bold')
    axes[0].axis('off')
    
    # Cartes Grad-CAM
    for i, result in enumerate(cam_results):
        # Superposition de la carte CAM sur l'image
        axes[i + 1].imshow(image, cmap='gray', alpha=0.7)
        
        # Carte de chaleur
        im = axes[i + 1].imshow(result['cam_map'], cmap='jet', alpha=0.6, 
                               vmin=0, vmax=1)
        
        # Titre avec pathologie et probabilité
        prob_percent = result['probability'] * 100
        status = "DÉTECTÉ" if result['probability'] > 0.5 else "Faible prob."
        color = 'red' if result['probability'] > 0.5 else 'orange'
        
        axes[i + 1].set_title(
            f"{result['pathology']}\n{prob_percent:.1f}% - {status}", 
            fontweight='bold', color=color, fontsize=10
        )
        axes[i + 1].axis('off')
        
        # Colorbar pour la première carte
        if i == 0:
            plt.colorbar(im, ax=axes[i + 1], fraction=0.046, pad=0.04, label='Importance')
    
    plt.suptitle(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Génération des visualisations Grad-CAM pour les cas cliniques
print("\nGénération des cartes de localisation Grad-CAM...")

if comprehensive_analysis and models_dict:
    # Sélection de quelques cas représentatifs
    selected_cases = ['pneumonia_lobaire', 'cardiomegalie_moderee', 'pneumothorax_spontane']
    
    for case_name in selected_cases:
        if case_name in comprehensive_analysis:
            case_data = comprehensive_analysis[case_name]
            processed_image = case_data['processed_image'][0]  # Premier canal
            
            print(f"\nGrad-CAM pour le cas: {case_data['case_info']['description']}")
            
            # Utilisation du premier modèle disponible pour Grad-CAM
            model_name = list(models_dict.keys())[0]
            model = models_dict[model_name]
            
            # Reconstruction du tenseur pour Grad-CAM
            image_tensor = torch.from_numpy(case_data['processed_image']).float().unsqueeze(0)
            
            try:
                # Génération Grad-CAM
                cam_results = generate_gradcam_visualization(
                    model, image_tensor, pathologies, top_k=3
                )
                
                if cam_results:
                    # Visualisation
                    visualize_gradcam_results(
                        processed_image, cam_results, 
                        f"Localisation - {case_data['case_info']['description']}"
                    )
                    
                    print(f"  Régions d'intérêt détectées:")
                    for result in cam_results:
                        if result['probability'] > 0.3:
                            print(f"    • {result['pathology']}: {result['probability']*100:.1f}%")
                else:
                    print(f"  Aucune localisation générée")
                
            except Exception as e:
                print(f"  Erreur Grad-CAM: {e}")

else:
    print("Données d'analyse ou modèles non disponibles pour Grad-CAM")

print("\nLocalisation spatiale terminée.")

## 5. Métriques de Performance et Comparaison de Modèles

Analysons en détail les performances des différents modèles.

In [ ]:
# Calcul des métriques de performance détaillées
print("=== MÉTRIQUES DE PERFORMANCE ET COMPARAISON ===")

def calculate_model_performance(comprehensive_analysis, models_info):
    """
    Calcule les métriques de performance pour chaque modèle
    """
    performance_metrics = {}
    
    # Initialisation des métriques par modèle
    for model_name in models_info.keys():
        performance_metrics[model_name] = {
            'true_positives': 0,
            'false_positives': 0,
            'false_negatives': 0,
            'total_detections': 0,
            'total_cases': 0,
            'probability_stats': {
                'max_probs': [],
                'mean_probs': [],
                'all_probs': []
            },
            'case_performances': []
        }
    
    # Calcul pour chaque cas
    for case_name, analysis in comprehensive_analysis.items():
        expected = analysis['case_info']['expected_pathologies']
        evaluation = analysis['evaluation']
        model_results = analysis['model_results']
        
        for model_name, result in model_results.items():
            if model_name in performance_metrics:
                metrics = performance_metrics[model_name]
                
                # Métriques de classification
                detected_pathologies = [p[0] for p in result['positive_findings']]
                
                # Calcul TP, FP, FN pour ce cas
                case_tp = 0
                case_fp = len(detected_pathologies)
                case_fn = len(expected)
                
                for exp_path in expected:
                    for det_path in detected_pathologies:
                        if (exp_path.lower() in det_path.lower() or 
                            det_path.lower() in exp_path.lower()):
                            case_tp += 1
                            case_fp -= 1
                            case_fn -= 1
                            break
                
                # Accumulation des métriques
                metrics['true_positives'] += case_tp
                metrics['false_positives'] += max(0, case_fp)
                metrics['false_negatives'] += max(0, case_fn)
                metrics['total_detections'] += result['num_detections']
                metrics['total_cases'] += 1
                
                # Statistiques de probabilité
                metrics['probability_stats']['max_probs'].append(result['max_probability'])
                metrics['probability_stats']['mean_probs'].append(result['mean_probability'])
                metrics['probability_stats']['all_probs'].extend(result['probabilities'])
                
                # Performance par cas
                metrics['case_performances'].append({
                    'case_name': case_name,
                    'tp': case_tp,
                    'fp': max(0, case_fp),
                    'fn': max(0, case_fn),
                    'detections': result['num_detections'],
                    'max_prob': result['max_probability']
                })
    
    # Calcul des métriques finales
    final_metrics = {}
    
    for model_name, metrics in performance_metrics.items():
        tp = metrics['true_positives']
        fp = metrics['false_positives']
        fn = metrics['false_negatives']
        
        # Métriques de classification
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        # Statistiques de probabilité
        prob_stats = metrics['probability_stats']
        
        final_metrics[model_name] = {
            'model_info': models_info[model_name],
            'classification_metrics': {
                'precision': precision,
                'recall': recall,
                'f1_score': f1_score,
                'true_positives': tp,
                'false_positives': fp,
                'false_negatives': fn
            },
            'detection_stats': {
                'avg_detections_per_case': metrics['total_detections'] / metrics['total_cases'],
                'total_detections': metrics['total_detections'],
                'total_cases': metrics['total_cases']
            },
            'probability_statistics': {
                'max_prob_mean': np.mean(prob_stats['max_probs']),
                'max_prob_std': np.std(prob_stats['max_probs']),
                'mean_prob_overall': np.mean(prob_stats['all_probs']),
                'prob_above_threshold': np.sum(np.array(prob_stats['all_probs']) > 0.5) / len(prob_stats['all_probs'])
            },
            'case_performances': metrics['case_performances']
        }
    
    return final_metrics

def create_performance_comparison_visualizations(final_metrics):
    """
    Crée des visualisations de comparaison de performance
    """
    if not final_metrics:
        print("Aucune métrique disponible pour visualisation")
        return
    
    model_names = list(final_metrics.keys())
    model_labels = [final_metrics[name]['model_info']['name'] for name in model_names]
    
    # Extraction des métriques
    precisions = [final_metrics[name]['classification_metrics']['precision'] for name in model_names]
    recalls = [final_metrics[name]['classification_metrics']['recall'] for name in model_names]
    f1_scores = [final_metrics[name]['classification_metrics']['f1_score'] for name in model_names]
    avg_detections = [final_metrics[name]['detection_stats']['avg_detections_per_case'] for name in model_names]
    
    # Création de la figure comparative
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Comparaison de Performance - Modèles TorchXRayVision', 
                fontsize=16, fontweight='bold')
    
    # 1. Métriques de classification
    x_pos = np.arange(len(model_labels))
    width = 0.25
    
    axes[0, 0].bar(x_pos - width, precisions, width, label='Précision', alpha=0.8, color='lightblue')
    axes[0, 0].bar(x_pos, recalls, width, label='Rappel', alpha=0.8, color='lightgreen')
    axes[0, 0].bar(x_pos + width, f1_scores, width, label='F1-Score', alpha=0.8, color='lightcoral')
    
    axes[0, 0].set_xticks(x_pos)
    axes[0, 0].set_xticklabels(model_labels, rotation=45, ha='right')
    axes[0, 0].set_ylabel('Score')
    axes[0, 0].set_title('Métriques de Classification')
    axes[0, 0].legend()
    axes[0, 0].set_ylim(0, 1)
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Détections moyennes par cas
    axes[0, 1].bar(model_labels, avg_detections, alpha=0.8, color='gold')
    axes[0, 1].set_title('Détections Moyennes par Cas')
    axes[0, 1].set_ylabel('Nombre de Détections')
    axes[0, 1].tick_params(axis='x', rotation=45)
    axes[0, 1].grid(True, alpha=0.3)
    
    # Ajout des valeurs sur les barres
    for i, v in enumerate(avg_detections):
        axes[0, 1].text(i, v + 0.1, f'{v:.1f}', ha='center', va='bottom', fontweight='bold')
    
    # 3. Matrice de confusion agrégée
    confusion_data = []
    for model_name in model_names:
        metrics = final_metrics[model_name]['classification_metrics']
        tp = metrics['true_positives']
        fp = metrics['false_positives']
        fn = metrics['false_negatives']
        confusion_data.append([tp, fp, fn])
    
    confusion_array = np.array(confusion_data)
    
    im = axes[1, 0].imshow(confusion_array.T, cmap='Blues', aspect='auto')
    axes[1, 0].set_xticks(range(len(model_labels)))
    axes[1, 0].set_xticklabels(model_labels, rotation=45, ha='right')
    axes[1, 0].set_yticks(range(3))
    axes[1, 0].set_yticklabels(['Vrais Positifs', 'Faux Positifs', 'Faux Négatifs'])
    axes[1, 0].set_title('Matrice de Confusion Agrégée')
    
    # Valeurs dans la matrice
    for i in range(len(model_labels)):
        for j in range(3):
            axes[1, 0].text(i, j, int(confusion_array[i, j]), ha="center", va="center", 
                           color="white", fontweight='bold')
    
    # 4. Radar plot des performances
    categories = ['Précision', 'Rappel', 'F1-Score']
    
    angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False)
    angles = np.concatenate((angles, [angles[0]]))  # Fermer le radar
    
    ax_radar = plt.subplot(2, 2, 4, projection='polar')
    
    colors = ['blue', 'red', 'green', 'orange'][:len(model_names)]
    
    for i, model_name in enumerate(model_names):
        values = [precisions[i], recalls[i], f1_scores[i]]
        values += values[:1]  # Fermer le radar
        
        ax_radar.plot(angles, values, 'o-', linewidth=2, label=model_labels[i], color=colors[i])
        ax_radar.fill(angles, values, alpha=0.25, color=colors[i])
    
    ax_radar.set_xticks(angles[:-1])
    ax_radar.set_xticklabels(categories)
    ax_radar.set_ylim(0, 1)
    ax_radar.set_title('Profil de Performance', y=1.08)
    ax_radar.legend(loc='upper right', bbox_to_anchor=(1.2, 1.0))
    ax_radar.grid(True)
    
    plt.tight_layout()
    plt.show()

# Calcul des métriques de performance
if comprehensive_analysis:
    print("\nCalcul des métriques de performance...")
    
    final_metrics = calculate_model_performance(comprehensive_analysis, models_info)
    
    # Affichage des résultats textuels
    print("\nRésumé des performances:")
    print("=" * 60)
    
    for model_name, metrics in final_metrics.items():
        model_info = metrics['model_info']
        class_metrics = metrics['classification_metrics']
        detection_stats = metrics['detection_stats']
        prob_stats = metrics['probability_statistics']
        
        print(f"\n{model_info['name']} ({model_info['description']}):")
        print(f"  Classification:")
        print(f"    • Précision: {class_metrics['precision']:.3f}")
        print(f"    • Rappel: {class_metrics['recall']:.3f}")
        print(f"    • F1-Score: {class_metrics['f1_score']:.3f}")
        print(f"  Détections:")
        print(f"    • Moyenne par cas: {detection_stats['avg_detections_per_case']:.1f}")
        print(f"    • Probabilité max moyenne: {prob_stats['max_prob_mean']:.3f}")
        print(f"    • % probabilités > 50%: {prob_stats['prob_above_threshold']*100:.1f}%")
    
    # Identification du meilleur modèle
    best_f1 = max(final_metrics.items(), key=lambda x: x[1]['classification_metrics']['f1_score'])
    best_precision = max(final_metrics.items(), key=lambda x: x[1]['classification_metrics']['precision'])
    best_recall = max(final_metrics.items(), key=lambda x: x[1]['classification_metrics']['recall'])
    
    print(f"\n=== RECOMMANDATIONS CLINIQUES ===")
    print(f"Meilleur F1-Score (équilibré): {final_metrics[best_f1[0]]['model_info']['name']}")
    print(f"Meilleure précision (éviter faux positifs): {final_metrics[best_precision[0]]['model_info']['name']}")
    print(f"Meilleur rappel (dépistage sensible): {final_metrics[best_recall[0]]['model_info']['name']}")
    
    # Création des visualisations
    create_performance_comparison_visualizations(final_metrics)
    
else:
    print("Aucune analyse disponible pour le calcul de métriques")

print("\nAnalyse de performance terminée.")

## 6. Génération Automatique de Rapports Cliniques

Créons un système de génération automatique de rapports radiologiques structurés.

In [ ]:
# Système de génération automatique de rapports cliniques
print("=== GÉNÉRATION AUTOMATIQUE DE RAPPORTS CLINIQUES ===")

class ClinicalReportGenerator:
    """
    Générateur de rapports radiologiques automatisés
    basé sur les résultats de classification TorchXRayVision
    """
    
    def __init__(self, pathology_descriptions=None):
        self.pathology_descriptions = pathology_descriptions or self._default_descriptions()
        self.clinical_recommendations = self._default_recommendations()
    
    def _default_descriptions(self):
        """
        Descriptions standardisées des pathologies
        """
        return {
            'Atelectasis': 'Atélectasie - Collapsus alvéolaire avec perte de volume pulmonaire',
            'Cardiomegaly': 'Cardiomégalie - Augmentation de la silhouette cardiaque',
            'Effusion': 'Épanchement pleural - Collection liquidienne dans l\'espace pleural',
            'Infiltration': 'Infiltrat pulmonaire - Opacités parenchymateuses diffuses',
            'Mass': 'Masse pulmonaire - Lésion focale de grande taille',
            'Nodule': 'Nodule pulmonaire - Lésion arrondie bien délimitée',
            'Pneumonia': 'Pneumonie - Consolidation alvéolaire d\'origine infectieuse',
            'Pneumothorax': 'Pneumothorax - Présence d\'air dans l\'espace pleural',
            'Consolidation': 'Consolidation - Comblement alvéolaire avec bronchogramme aérique',
            'Edema': 'Œdème pulmonaire - Accumulation de liquide dans les alvéoles',
            'Emphysema': 'Emphysème - Destruction des parois alvéolaires',
            'Fibrosis': 'Fibrose pulmonaire - Épaississement cicatriciel du parenchyme',
            'Pleural Thickening': 'Épaississement pleural - Fibrose de la plèvre pariétale',
            'Hernia': 'Hernie - Déplacement d\'organes à travers un orifice'
        }
    
    def _default_recommendations(self):
        """
        Recommandations cliniques par pathologie
        """
        return {
            'Pneumonia': {
                'urgency': 'Modérée à élevée',
                'follow_up': 'Contrôle radiologique à 48-72h sous traitement',
                'specialty': 'Pneumologie si évolution défavorable',
                'additional_tests': 'Hémocultures, ECBU si fièvre, CRP, PCT'
            },
            'Pneumothorax': {
                'urgency': 'Élevée',
                'follow_up': 'Surveillance clinique rapprochée, contrôle radiologique',
                'specialty': 'Pneumologie en urgence si > 20%',
                'additional_tests': 'Gazométrie artérielle si détresse respiratoire'
            },
            'Cardiomegaly': {
                'urgency': 'Modérée',
                'follow_up': 'Échocardiographie, contrôle à 3 mois',
                'specialty': 'Cardiologie pour bilan étiologique',
                'additional_tests': 'ECG, BNP, bilan biologique cardiaque'
            },
            'Mass': {
                'urgency': 'Élevée',
                'follow_up': 'Scanner thoracique avec injection urgente',
                'specialty': 'Pneumologie/Oncologie pour RCP',
                'additional_tests': 'Marqueurs tumoraux, fibroscopie bronchique'
            },
            'Nodule': {
                'urgency': 'Modérée',
                'follow_up': 'Scanner thoracique, contrôle selon taille',
                'specialty': 'Pneumologie pour caractérisation',
                'additional_tests': 'TEP-scan si nodule > 8mm'
            }
        }
    
    def generate_structured_report(self, case_analysis, patient_info=None):
        """
        Génère un rapport radiologique structuré
        """
        case_info = case_analysis['case_info']
        consensus = case_analysis['consensus']
        evaluation = case_analysis['evaluation']
        
        # En-tête du rapport
        report = self._generate_header(patient_info)
        
        # Technique d'examen
        report += self._generate_technique_section()
        
        # Résultats
        report += self._generate_findings_section(consensus, evaluation)
        
        # Impression
        report += self._generate_impression_section(consensus)
        
        # Recommandations
        report += self._generate_recommendations_section(consensus)
        
        # Signature
        report += self._generate_signature_section()
        
        return report
    
    def _generate_header(self, patient_info):
        """
        Génère l'en-tête du rapport
        """
        from datetime import datetime
        
        patient_id = patient_info.get('id', 'DEMO-001') if patient_info else 'DEMO-001'
        patient_name = patient_info.get('name', 'Patient Démonstration') if patient_info else 'Patient Démonstration'
        
        return f"""
RAPPORT RADIOLOGIQUE
====================================

Patient: {patient_name}
ID: {patient_id}
Examen: Radiographie thoracique de face
Date: {datetime.now().strftime('%d/%m/%Y')}
Heure: {datetime.now().strftime('%H:%M')}

Système d'analyse: TorchXRayVision (IA)
Statut: RAPPORT AUTOMATISÉ - Validation médicale requise

"""
    
    def _generate_technique_section(self):
        """
        Section technique
        """
        return """
TECHNIQUE:
Radiographie thoracique numérique de face en position debout.
Inspiration correcte, positionnement satisfaisant.
Analyse par intelligence artificielle avec modèles DenseNet.

"""
    
    def _generate_findings_section(self, consensus, evaluation):
        """
        Section des résultats
        """
        findings = "RÉSULTATS:\n"
        
        if not consensus or len(consensus) == 0:
            findings += "Examen dans les limites de la normale.\n"
            findings += "Aucune anomalie significative détectée par l'analyse IA.\n"
        else:
            findings += "Anomalies détectées par analyse IA:\n\n"
            
            for i, finding in enumerate(consensus[:5], 1):  # Top 5 findings
                pathology = finding['pathology']
                consensus_level = finding['consensus_level']
                avg_prob = finding['avg_probability']
                
                # Description de la pathologie
                description = self.pathology_descriptions.get(
                    pathology, f"Anomalie détectée: {pathology}"
                )
                
                # Niveau de confiance
                if consensus_level >= 0.8:
                    confidence = "Confiance élevée"
                elif consensus_level >= 0.5:
                    confidence = "Confiance modérée"
                else:
                    confidence = "Confiance limitée"
                
                findings += f"{i}. {description}\n"
                findings += f"   - Probabilité: {avg_prob:.1%}\n"
                findings += f"   - Consensus inter-modèles: {consensus_level:.1%}\n"
                findings += f"   - {confidence}\n\n"
        
        # Évaluation de performance
        if evaluation['detected_expected']:
            findings += "Concordance avec attentes cliniques:\n"
            for detected in evaluation['detected_expected']:
                findings += f"✓ {detected}\n"
        
        if evaluation['missed_expected']:
            findings += "\nPathologies attendues non détectées:\n"
            for missed in evaluation['missed_expected']:
                findings += f"? {missed} (possibilité de faux négatif)\n"
        
        return findings + "\n"
    
    def _generate_impression_section(self, consensus):
        """
        Section impression diagnostique
        """
        impression = "IMPRESSION:\n"
        
        if not consensus or len(consensus) == 0:
            impression += "Radiographie thoracique normale.\n"
        else:
            # Pathologies principales (consensus > 50%)
            main_findings = [f for f in consensus if f['consensus_level'] > 0.5]
            
            if main_findings:
                impression += "Anomalies principales détectées:\n"
                for finding in main_findings:
                    pathology = finding['pathology']
                    probability = finding['avg_probability']
                    
                    # Classification de sévérité basée sur probabilité
                    if probability > 0.8:
                        severity = "probable"
                    elif probability > 0.6:
                        severity = "possible"
                    else:
                        severity = "à confirmer"
                    
                    impression += f"- {pathology} {severity} ({probability:.1%})\n"
            
            # Pathologies secondaires
            secondary_findings = [f for f in consensus if f['consensus_level'] <= 0.5]
            if secondary_findings:
                impression += "\nAnomalies mineures ou incertaines:\n"
                for finding in secondary_findings[:3]:
                    pathology = finding['pathology']
                    impression += f"- {pathology} (incertain)\n"
        
        return impression + "\n"
    
    def _generate_recommendations_section(self, consensus):
        """
        Section recommandations
        """
        recommendations = "RECOMMANDATIONS:\n"
        
        if not consensus or len(consensus) == 0:
            recommendations += "Aucune recommandation spécifique.\n"
            recommendations += "Contrôle selon indication clinique.\n"
        else:
            # Recommandations spécifiques par pathologie
            high_priority = []
            
            for finding in consensus:
                if finding['consensus_level'] > 0.5:
                    pathology = finding['pathology']
                    
                    # Recherche de recommandations spécifiques
                    for key, rec in self.clinical_recommendations.items():
                        if key.lower() in pathology.lower():
                            recommendations += f"\n{pathology}:\n"
                            recommendations += f"- Urgence: {rec['urgency']}\n"
                            recommendations += f"- Suivi: {rec['follow_up']}\n"
                            recommendations += f"- Spécialiste: {rec['specialty']}\n"
                            recommendations += f"- Examens complémentaires: {rec['additional_tests']}\n"
                            
                            if rec['urgency'] == 'Élevée':
                                high_priority.append(pathology)
                            break
            
            # Recommandations générales
            recommendations += "\nRecommandations générales:\n"
            if high_priority:
                recommendations += "⚠️ PRISE EN CHARGE URGENTE recommandée\n"
                recommendations += "📞 Contacter le spécialiste en priorité\n"
            
            recommendations += "🔬 Corrélation avec l'examen clinique indispensable\n"
            recommendations += "👨‍⚕️ Validation par radiologue senior recommandée\n"
        
        return recommendations + "\n"
    
    def _generate_signature_section(self):
        """
        Section signature
        """
        from datetime import datetime
        
        return f"""
VALIDATION:
Rapport généré automatiquement par système d'IA TorchXRayVision
Date de génération: {datetime.now().strftime('%d/%m/%Y à %H:%M')}

⚠️  IMPORTANT: Ce rapport automatisé doit être validé par un radiologue.
⚠️  L'IA est un outil d'aide au diagnostic, non substitutif à l'expertise médicale.
⚠️  En cas de doute clinique, privilégier l'avis du radiologue senior.

Système: TorchXRayVision Clinical Assistant
Version: Démonstration pédagogique
Statut: NON VALIDÉ - RAPPORT ÉDUCATIF
"""

# Démonstration de génération de rapports
print("\nGénération de rapports cliniques automatisés...")

if comprehensive_analysis:
    # Initialisation du générateur
    report_generator = ClinicalReportGenerator()
    
    # Génération de rapports pour quelques cas sélectionnés
    selected_cases = ['pneumonia_lobaire', 'cardiomegalie_moderee', 'normal']
    
    for case_name in selected_cases:
        if case_name in comprehensive_analysis:
            print(f"\n{'='*80}")
            print(f"RAPPORT AUTOMATISÉ - CAS: {case_name.upper()}")
            print(f"{'='*80}")
            
            # Informations patient fictives
            patient_info = {
                'id': f'DEMO-{case_name[:3].upper()}-001',
                'name': f'Patient {case_name.replace("_", " ").title()}'
            }
            
            # Génération du rapport
            case_analysis = comprehensive_analysis[case_name]
            clinical_report = report_generator.generate_structured_report(
                case_analysis, patient_info
            )
            
            print(clinical_report)
    
    print(f"\nRapports cliniques générés pour {len(selected_cases)} cas.")
    print("Ces rapports démontrent l'intégration possible de TorchXRayVision")
    print("dans des workflows cliniques automatisés.")

else:
    print("Aucune analyse disponible pour génération de rapports")

print("\nGénération de rapports terminée.")

## Résumé et Applications Cliniques

### Compétences Acquises

Dans ce notebook complet, vous avez maîtrisé:

1. **Analyse Multi-Modèles avec TorchXRayVision**
   - Chargement et comparaison de modèles DenseNet-All, CheXpert, NIH
   - Classification de 18 pathologies thoraciques principales
   - Analyse de consensus inter-modèles pour robustesse diagnostique

2. **Création de Cas Cliniques Réalistes**
   - Génération de radiographies synthétiques pathologiques
   - Simulation de conditions cliniques variées
   - Intégration de contextes médicaux authentiques

3. **Localisation Spatiale avec Grad-CAM**
   - Implémentation de cartes d'activation pour localisation
   - Visualisation des régions d'intérêt pathologiques
   - Explicabilité des décisions d'IA pour validation clinique

4. **Métriques de Performance Avancées**
   - Calcul de précision, rappel, F1-Score par modèle
   - Analyse comparative des performances diagnostiques
   - Recommandations d'usage selon contexte clinique

5. **Génération Automatique de Rapports**
   - Rapports radiologiques structurés automatisés
   - Recommandations cliniques spécialisées par pathologie
   - Intégration dans workflows hospitaliers

### Applications Médicales Directes

Ces compétences vous permettront de:
- **Déployer TorchXRayVision** dans des environnements cliniques réels
- **Optimiser la sélection de modèles** selon vos besoins spécifiques
- **Intégrer l'IA radiologique** dans les systèmes PACS existants
- **Générer des rapports automatisés** pour améliorer l'efficacité
- **Localiser précisément** les pathologies pour guidage interventionnel

### Recommandations Cliniques

1. **Sélection de Modèles**:
   - **DenseNet-All**: Usage général, meilleur équilibre performance
   - **CheXpert**: Pathologies thoraciques variées, haute spécificité
   - **NIH**: Conditions spécialisées, recherche clinique

2. **Intégration Workflow**:
   - Utiliser le consensus inter-modèles pour validation
   - Implémenter des seuils adaptatifs selon urgence
   - Maintenir la supervision médicale obligatoire

3. **Assurance Qualité**:
   - Validation continue sur données locales
   - Surveillance des performances en temps réel
   - Formation continue des équipes médicales

### Limitations et Considérations Éthiques

- **Validation Médicale**: L'IA reste un outil d'aide, jamais substitutif
- **Biais des Données**: Performance variable selon populations
- **Évolution Technologique**: Mise à jour régulière nécessaire
- **Responsabilité Clinique**: Décision finale toujours médicale

### Prochaine Étape

Le prochain notebook vous enseignera la **segmentation d'images médicales**, combinant méthodes manuelles et automatiques avec MedSAM pour délimitation précise des structures anatomiques.